# __MODIS vs THAAO comparison of Integrated Water Vapor (IWV)__
[MODIS](https://modis.gsfc.nasa.gov/about/specifications.php): Moderate Resolution Imaging Spectroradiometer  <br>
[THAAO](https://www.thuleatmos-it.it/index.php): Thule High Arctic Atmospheric Observatory, Greenland  (coordinates  __76.5145, -68.7432__, altitude a.s.l. __220 m__)  <br>

__MXD05_L2 IWV product__ <br>
_References_ <br>
[MYD05_L2](https://atmosphere-imager.gsfc.nasa.gov/products/water-vapor) <br>

_Algorithm Theoretical Basis Documents (ATBDs)_ <br>
[ATBD](https://atmosphere-imager.gsfc.nasa.gov/documentation/atbds-plans-guides) <br>
[The MODIS Near-IR Water Vapor Algorithm](https://atmosphere-imager.gsfc.nasa.gov/sites/default/files/ModAtmo/atbd_mod03_0.pdf) <br>
[MODIS Atmospheric Profile Retrieval
ATBD MOD07 Collection 6](https://modis-images.gsfc.nasa.gov/_docs/MOD07_atbd_v7_April2011.pdf) <br>
[Atmospheric Correction Algorithm: Spectral
Reflectances MOD08](https://modis.gsfc.nasa.gov/data/atbd/atbd_mod08.pdf) <br>
[MODIS Atmosphere L3 Gridded Product
(ATBD) & Users Guide](https://atmosphere-imager.gsfc.nasa.gov/sites/default/files/ModAtmo/L3_ATBD_C6_C61_2019_02_20.pdf) <br>

__THAAO IWV product__ <br>
_References_ <br>
[The water Vapor Emission Spectrometer for Polar Atmospheres at 22 GHz](https://www.thuleatmos-it.it/data/vespa22/index.php)<br> 
[Integrated Water Vapor](https://www.thuleatmos-it.it/dataaccess/vapor/index.php)<br>
 

In [ ]:
%run reader.ipynb

#### __MXD05_L2 and THAAO IWV products__ 

Satellite MODIS MXD05_L2 data products covering the Greenland region on a time interval spanning from July 2016 to January 2022 were first downloaded from the LAADS repository by means of custom Python scripts. Timeseries of IWV IR and ancillary data products were then extracted on a point geometry centered on the THAAO Observatory, Greenland from the MXD05_L2 products and packed together as csv files for further analisys. <br>

Thule (Greenland) IWV data timeseries have been downloaded from the [Integrated Water Vapor](https://www.thuleatmos-it.it/dataaccess/vapor/index.php) THAAO data access facility were the data are freely accessible. Likewise MODIS satellite IWV product, these data contain vertical column integrated amount  water vapor or precipitable water retrieved by means of the water Vapor Emission Spectrometer for Polar Atmospheres at 22 GHz (VESPA-22).  <br> 


#### __Loading MODIS and Thule timeseries__  
IWV timeseries of MODIS in .csv and THAAO in .txt formats are read into pandas dataframes. <br>
MODIS Near-IR product has an estimated additional 13% error column derived from the maximum error claimed in the NearIR IWV product ATBD (see sect. 3.1.4, page 16). <br>
THAAO IWV data are given in $kg/m^2$ units and thus divided for factor of 10 to have homogeoeous units with MODIS IWV data that are given in $cm$. <br>  

In [ ]:
mod_data = modis_read('MODIS/MOD05_5km_totale.csv')
myd_data = modis_read('MODIS/MYD05_5km_totale.csv')
vespa_data = vespa_read('VESPA/Vapor_20160716_20220311.txt')

In [ ]:
vespa_data.tail()

In [ ]:
myd_data.tail()

### Drop rows with IWV values > 50 mm

In [ ]:
mod_data = df_drop(mod_data, mod_data.IWV_MODIS > 50.)
myd_data = df_drop(myd_data, myd_data.IWV_MODIS > 50. )

In [ ]:
mod_data = distance_from_station(mod_data,drop_above=10.)
myd_data = distance_from_station(myd_data,drop_above=10.)

In [ ]:
number_mxd_data_points = len(mod_data.IWV_MODIS) + len(myd_data.IWV_MODIS)
number_mxd_data_points

#### **MODIS and THAAO Integrated Water Vapor (IWV) timeseries**
##### Simple linear plots of the whole MODIS and THAAO IVW timeseries.

In [ ]:
mod_data['Err_IWV_MODIS']

In [ ]:
# visualization options
width = 1100
height = 300 
size = 3
scale = 0.3
color_MOD = 'cyan'
color_MYD = '#F2583E'
color_THAAO = 'green'

alpha=100
dynspread=False
datashade=False
legend='top_right'

# text options
xlabel='Time'
ylabel='IWV    [mm]'

plot_defaults = dict(xlabel=xlabel, ylabel=ylabel,height=height,
                width=width,alpha=alpha,dynspread=dynspread,
                datashade=datashade,size=size, legend=legend)

In [ ]:
#(mod_data.IWV_MODIS.hvplot.scatter(xlabel='Time', ylabel='IWV    [mm]', 
#                                 height=height, width=width, size=size, 
#                                 color=color_MOD, title ='IWV MOD')
(mod_data.IWV_MODIS.hvplot.scatter(color=color_MOD, title ='IWV MOD', **plot_defaults)
* mod_data.hvplot.errorbars(y='IWV_MODIS', yerr1='Err_IWV_MODIS', hover=False)
+ myd_data.IWV_MODIS.hvplot.scatter(xlabel='Time', ylabel='IWV    [mm]', 
                                  height=height, width=width, size=size, 
                                  color=color_MYD,title ='IWV MYD')
* myd_data.hvplot.errorbars(y='IWV_MODIS', yerr1='Err_IWV_MODIS', hover=False)
+ (mod_data.IWV_MODIS.hvplot.scatter(xlabel='Time', ylabel='IWV    [mm]', 
                                   height=height, width=width, size=size, 
                                   color=color_MOD,title ='IWV MOD')
* mod_data.hvplot.errorbars(y='IWV_MODIS', yerr1='Err_IWV_MODIS', hover=False)
* myd_data.IWV_MODIS.hvplot.scatter(xlabel='Time', ylabel='IWV    [mm]', 
                                  height=height, width=width, size=size, 
                                  color=color_MYD,title ='IWV MYD')
* myd_data.hvplot.errorbars(y='IWV_MODIS', yerr1='Err_IWV_MODIS', hover=False)
  ).opts(title='MOD+MYD')).cols(1)

In [ ]:
#MOD_IR #['Err_IWV_MODIS']

In [ ]:
thaao_timeseries = vespa_data.IWV_THAAO.hvplot.scatter(yaxis=True, 
                                        height=height, width=width, 
                                        size=size, 
                                        color=color_THAAO, 
                                        fields={'date': 'Date'},
                                        title='IWV THAAO - VESPA', ylabel='VESPA    [mm]',
                                        legend='top_right')
thaao_timeseries

In [ ]:
# TODO: plot con errori di VESPA ha un errore: gli errori sono troppo grandi, non so perché:
# vedi min, max e cella dopo con vespa_data.head()
print(f'min_err = {vespa_data.err_IWV.min()}, max_err = {vespa_data.err_IWV.max()}')
thaao_timeseries*vespa_data.hvplot.errorbars(y='IWV_THAAO', yerr1='err_IWV', hover=False)

In [ ]:
vespa_data.head()

Overlay of the IWV plots on the same time reference <br>

In [ ]:
mod_timeseries = mod_data.IWV_MODIS.hvplot.scatter(xlabel='Time', 
                                                 ylabel='IWV   [cm]',label='MOD',
                                                 height=height, width=width, 
                                                 size=size, color=color_MOD, 
                                                 title='IWV MOD - VESPA',
                                                 legend='top_right')
myd_timeseries = myd_data.IWV_MODIS.hvplot.scatter(xlabel='Time', ylabel='IWV   [cm]',label='MYD',
                                                 height=height, width=width, 
                                                 size=size, color=color_MYD, 
                                                 title='IWV MYD - VESPA',
                                                 legend='top_right')


(mod_timeseries *thaao_timeseries
+ myd_timeseries * thaao_timeseries
+ mod_timeseries*myd_timeseries * thaao_timeseries
* mod_data.hvplot.errorbars(y='IWV_MODIS', yerr1='Err_IWV_MODIS', hover=False)
* myd_data.hvplot.errorbars(y='IWV_MODIS', yerr1='Err_IWV_MODIS', hover=False)).cols(1)

In [ ]:
title = f'IWV MXD-VESPA                                         # data points={number_mxd_data_points}'
(mod_timeseries*myd_timeseries * thaao_timeseries).opts(title=title, show_grid=True)

### __TODO filtrare i dati MODIS con il QA__ !!!

#### __Merging pandas dataframes to filter MODIS and THAAO data points closest in time__

Due to time lenght of the timeseries, even if the notebook representation allows to zoom in into the datasets, it is still not easy to see how close in time two data points are. Since it makes little sense to compare two MODIS and THAAO IWV data points if they are too far apart, and they can easily be hours or days far apart, therefore easily mistaken if one relies solely on the visual inspection of the plots, a tool to select and compare only the data points which are closer in time is required. The following code allows to filter the data on a timespan basis by suitably merging modis and thaao pandas dataframes while defining the Timedelta parameter (expressed in s,m,d, i.e. seconds, minutes or days respectively) that allows to operate the time-based filter. Different choices of Timedelta allow to widen or narrow the time span around each MODIS data point and select for the comparison only the THAAO data points that fall within the chosen interval. This selection is based on MODIS data points because THAAO measures are far more dense. <br>  

In [ ]:
# processing and visualization parameters
deltatime = '10m'

In [ ]:
new_myd_vespa = closest_in_time2(myd_data,vespa_data,'IWV_MODIS','IWV_THAAO',on='Time',delta_time=deltatime)
new_mod_vespa = closest_in_time2(mod_data,vespa_data,'IWV_MODIS','IWV_THAAO',on='Time',delta_time=deltatime)

In [ ]:
# number_mxd_data_points for the selected deltatime 
num_mxd_vespa_points_deltatime = len(new_mod_vespa.IWV_MODIS) + len(new_myd_vespa.IWV_MODIS)
num_mxd_vespa_points_deltatime, deltatime

In [ ]:
new_mod_vespa.head()
new_mod_vespa.vza.max()

In [ ]:
new_myd_vespa.tail()

## controllo distanza e lat, lon

In [ ]:
new_myd_vespa.diff_distance.min(), new_myd_vespa.diff_distance.max()

In [ ]:
new_myd_vespa.latitude.min(), new_myd_vespa.latitude.max(), new_myd_vespa.longitude.min(), new_myd_vespa.longitude.max()

In [ ]:
new_mod_vespa.diff_distance.min(), new_mod_vespa.diff_distance.max()

In [ ]:
new_mod_vespa.latitude.min(), new_mod_vespa.latitude.max(), new_mod_vespa.longitude.min(), new_mod_vespa.longitude.max()

In [ ]:
new_mxd_vespa = pd.concat([new_mod_vespa,new_myd_vespa], sort=True)
new_mxd_vespa.tail()

### Numero punti per mese 

In [ ]:
new_mod_vespa.Time.groupby([new_mod_vespa['Time'].dt.month.rename('month')]).agg({'count'}).transpose()

In [ ]:
new_myd_vespa.Time.groupby([new_myd_vespa['Time'].dt.month.rename('month')]).agg({'count'}).transpose()

### Timeseries su diff temporali 

In [ ]:

new_mod_vespa_timeseries = new_mod_vespa.IWV_MODIS.hvplot.scatter(x='Time_IWV_MODIS', 
                                                          xlabel='Time',
                                                          ylabel='IWV (cm)',
                                                          label='MOD',
                                                          height=height, 
                                                          width=width, 
                                                          size=size, 
                                                          color=color_MOD, 
                                                          title= 'IWV MOD-VESPA',
                                                          legend='top_right')

new_myd_vespa_timeseries = new_myd_vespa.IWV_MODIS.hvplot.scatter(x='Time_IWV_MODIS',
                                                          xlabel='Time',
                                                          ylabel='IWV (cm)',
                                                          label='MYD',
                                                          height=height, 
                                                          width=width, 
                                                          size=size, 
                                                          color=color_MYD, 
                                                          title= 'IWV MYD-VESPA',
                                                          legend='top_right')

(new_mod_vespa_timeseries*new_mod_vespa.IWV_THAAO.hvplot.scatter(x='Time_IWV_THAAO',
                                                         label='VESPA',                                                        
                                                         height=height, 
                                                         width=width, 
                                                         size=size, 
                                                         color=color_THAAO)
+ new_myd_vespa_timeseries*new_myd_vespa.IWV_THAAO.hvplot.scatter(x='Time_IWV_THAAO',
                                                          label='VESPA',
                                                          height=height, 
                                                          width=width, 
                                                          size=size, 
                                                          color=color_THAAO)
+ (new_mod_vespa_timeseries *new_myd_vespa_timeseries
* new_mod_vespa.IWV_THAAO.hvplot.scatter(x='Time_IWV_THAAO', height=height, 
                                     width=width, size=size, 
                                     label='VESPA',                                                        
                                     color=color_THAAO)
* new_myd_vespa.IWV_THAAO.hvplot.scatter(x='Time_IWV_THAAO',height=height, 
                                     width=width, size=size, 
                                     label='VESPA',                                                        
                                     color=color_THAAO)).opts(title='IWV MXD-VESPA')
+ (new_mod_vespa_timeseries *new_myd_vespa_timeseries
* new_mod_vespa.hvplot.errorbars(y='IWV_MODIS', yerr1='Err_IWV_MODIS', hover=False)
* new_myd_vespa.hvplot.errorbars(y='IWV_MODIS', yerr1='Err_IWV_MODIS', hover=False)   
* new_mod_vespa.IWV_THAAO.hvplot.scatter(x='Time_IWV_THAAO', height=height, 
                                     width=width, size=size, 
                                     label='VESPA',                                                        
                                     color=color_THAAO)
* new_myd_vespa.IWV_THAAO.hvplot.scatter(x='Time_IWV_THAAO',height=height, 
                                     width=width, size=size, 
                                     label='VESPA',                                                        
                                     color=color_THAAO)).opts(title='IWV MXD-VESPA')).cols(1)

In [ ]:
new_mod_vespa.head(1)

In [ ]:
# ok
title = f'IWV MXD-VESPA              deltatime={deltatime}              # data points={num_mxd_vespa_points_deltatime}'
(new_mod_vespa_timeseries * new_myd_vespa_timeseries 
* new_mod_vespa.IWV_THAAO.hvplot.scatter(x='Time_IWV_THAAO', height=height, 
                                     width=width, size=size, 
                                     label='VESPA',                                                        
                                     color=color_THAAO)
* new_myd_vespa.IWV_THAAO.hvplot.scatter(x='Time_IWV_THAAO',height=height, 
                                     width=width, size=size, 
                                     label='VESPA',                                                        
                                     color=color_THAAO)).opts(title=title, show_grid=True)

In [ ]:
# TODO non fa vedere la legenda di MYD
(mod_data.IWV_MODIS.hvplot.scatter(xlabel='Time_IWV_MODIS', ylabel='IWV   [cm]', 
                                   height=height, width=width, size=size, 
                                   color=color_MOD,title ='IWV MOD',
                                   label='MOD',
                                   legend='top_right')
* mod_data.hvplot.errorbars(y='IWV_MODIS', yerr1='Err_IWV_MODIS', hover=False)
* myd_data.IWV_MODIS.hvplot.scatter(xlabel='Time_IWV_MODIS', ylabel='IWV   [cm]', 
                                  height=height, width=width, size=size, 
                                  color=color_MYD,title ='IWV MYD',
                                  label='MYD')
* myd_data.hvplot.errorbars(y='IWV_MODIS', yerr1='Err_IWV_MODIS', hover=False)
  ).opts(title='IWV MXD', show_grid=True) 

### bisettrice

In [ ]:
slope = hv.Slope(1, 0).opts(line_color='red')

### Regressione con Holoviews

### **Regressione con sklearn**

In [ ]:
regress_mod_sk = regression_sklearn(new_mod_vespa.IWV_MODIS,new_mod_vespa.IWV_THAAO)
regress_mod_sk_text = regression_eq_text(new_mod_vespa.IWV_MODIS,
                                         new_mod_vespa.IWV_THAAO, regress_mod_sk)
regress_myd_sk = regression_sklearn(new_myd_vespa.IWV_MODIS,
                                    new_myd_vespa.IWV_THAAO, color='black')
regress_myd_sk_text = regression_eq_text(new_myd_vespa.IWV_MODIS,
                                         new_myd_vespa.IWV_THAAO, regress_myd_sk)
regress_mxd_vespa_sk = regression_sklearn(new_mxd_vespa.IWV_MODIS,new_mxd_vespa.IWV_THAAO, 
                                    color='cyan')
regress_mxd_vespa_sk_text = regression_eq_text(new_mxd_vespa.IWV_MODIS,
                                         new_mxd_vespa.IWV_THAAO, regress_mxd_vespa_sk)

### **MODIS vs THAAO Integrated Water Vapor (IWV) timeseries scatter plots**

### Scatter dimensione punti su differenza secondi

In [ ]:
scale = 0.5
alpha = 0.7
vespaVSmod = new_mod_vespa.hvplot.scatter(x='IWV_MODIS', y='IWV_THAAO',
                                      height=500, width=600, c='vza',
                                      s='diff_int', scale=scale,
                                      alpha=alpha,
                                      xlabel='IWV MOD    [mm]', 
                                      ylabel='IWV THAAO    [mm]'
                                     ).hist(num_bins=21,dimension=['IWV_MODIS','IWV_THAAO'])


vespaVSmyd = new_myd_vespa.hvplot.scatter(x='IWV_MODIS', y='IWV_THAAO', 
                                      height=500, width=600, c='vza',
                                      s='diff_int', scale=scale, 
                                      alpha=alpha,
                                      xlabel='IWV MYD    [mm]', 
                                      ylabel='IWV THAAO    [mm]'
                                     ).hist(num_bins=21,dimension=['IWV_MODIS','IWV_THAAO'])

(vespaVSmod*slope*regress_mod_sk[0]*regress_mod_sk_text
+ vespaVSmyd*slope*regress_myd_sk[0]*regress_myd_sk_text
+ vespaVSmod*vespaVSmyd*slope*regress_mxd_vespa_sk_text
* regress_mod_sk[0]*regress_myd_sk[0]*regress_mxd_vespa_sk[0]).cols(2)

### Scatter dimensione punti su differenza distanza geografica

In [ ]:
# diff_distance non viene calcolata in closest_in_time2() perché varia poco e variano poco i risultati, 
# quindi la cella va in errore. In caso di verifica reinserire la colonna 'diff_distance' nel dataframe  

scale = 5
vespaVSmod_scatter = new_mod_vespa.hvplot.scatter(x='IWV_MODIS', y='IWV_THAAO',
                                              height=500, width=600, c='vza',
                                              s='diff_distance', scale=scale,
                                              xlabel='IWV MOD    [mm]', 
                                              ylabel='IWV THAAO    [mm]'
                                             ).hist(num_bins=21,dimension=['IWV_MODIS','IWV_THAAO'])

vespaVSmyd_scatter = new_myd_vespa.hvplot.scatter(x='IWV_MODIS', y='IWV_THAAO',
                                              height=500, width=600, c='vza',
                                              s='diff_distance', scale=scale,
                                              xlabel='IWV MYD    [mm]',
                                              ylabel='IWV THAAO    [mm]'
                                             ).hist(num_bins=21,dimension=['IWV_MODIS','IWV_THAAO'])

(vespaVSmod_scatter*slope*regress_mod_sk[0]*regress_mod_sk_text
+ vespaVSmyd_scatter*slope*regress_myd_sk[0]*regress_myd_sk_text
+ vespaVSmod_scatter*vespaVSmyd_scatter*slope*regress_mxd_vespa_sk_text
 *regress_mod_sk[0]*regress_myd_sk[0]*regress_mxd_vespa_sk[0]).opts(width=400, height=300).cols(1)

### Scatter totale per piattaforma

In [ ]:
# diff_distance non viene calcolata in closest_in_time2() perché varia poco e variano poco i risultati, 
# quindi la cella va in errore. In caso di verifica reinserire la colonna 'diff_distance' nel dataframe  
scale = 5
vespaVSmod_scatter = new_mod_vespa.hvplot.scatter(x='IWV_MODIS', y='IWV_THAAO',
                                              height=height, width=width, 
                                              c='vza',
                                              s='diff_distance', scale=scale,
                                              xlabel='IWV MOD    [mm]',
                                              ylabel='IWV THAAO    [mm]'
                                             ).hist(num_bins=21,dimension=['IWV_MODIS','IWV_THAAO'])


vespaVSmyd_scatter = new_myd_vespa.hvplot.scatter(x='IWV_MODIS', y='IWV_THAAO',
                                              height=height, width=width, 
                                              c='vza',
                                              s='diff_distance', scale=scale, 
                                              xlabel='IWV MYD    [mm]', 
                                              ylabel='IWV THAAO    [mm]'
                                             ).hist(num_bins=21,dimension=['IWV_MODIS','IWV_THAAO'])

(vespaVSmod_scatter*slope*regress_mod_sk[0]*regress_mod_sk_text
+ vespaVSmyd_scatter*slope*regress_myd_sk[0]*regress_myd_sk_text
+ vespaVSmod_scatter*vespaVSmyd_scatter*slope*regress_mxd_vespa_sk_text
 *regress_mod_sk[0]*regress_myd_sk[0]*regress_mxd_vespa_sk[0]).cols(1)

In [ ]:
# ok
height=500
width=700
scale = 5
title = f'IWV MXD-VESPA              deltatime={deltatime}         # data points={num_mxd_vespa_points_deltatime}'

vespaVSmod_scatter_ok = new_mod_vespa.hvplot.scatter(x='IWV_MODIS', y='IWV_THAAO',
                                              height=height, width=width, 
                                              c='vza',
                                              s='diff_distance', scale=scale,
                                              xlabel='IWV MOD    [mm]',
                                              ylabel='IWV VESPA    [mm]')

vespaVSmyd_scatter_ok = new_myd_vespa.hvplot.scatter(x='IWV_MODIS', y='IWV_THAAO',
                                              height=height, width=width, 
                                              c='vza',
                                              s='diff_distance', scale=scale, 
                                              xlabel='IWV MYD    [mm]', 
                                              ylabel='IWV VESPA    [mm]')

(vespaVSmod_scatter_ok*vespaVSmyd_scatter_ok
*slope
*regress_mxd_vespa_sk_text
#*regress_MOD_sk[0]*regress_MYD_sk[0]
*regress_mxd_vespa_sk[0]).opts(title=title, show_grid=True, xlabel= 'IWV MXD    [mm]')

In [ ]:
# ok
height=500
width=700
scale = 1
title = f'IWV MXD-VESPA              deltatime={deltatime}         # data points={num_mxd_vespa_points_deltatime}'

vespaVSmod_scatter_ok = new_mod_vespa.hvplot.scatter(x='IWV_MODIS', y='IWV_THAAO',
                                              height=height, width=width, 
                                              c='vza',
                                              s='vza', scale=scale,
                                              xlabel='IWV MOD    [mm]',
                                              ylabel='IWV VESPA    [mm]')

vespaVSmyd_scatter_ok = new_myd_vespa.hvplot.scatter(x='IWV_MODIS', y='IWV_THAAO',
                                              height=height, width=width, 
                                              c='vza',
                                              s='vza', scale=scale, 
                                              xlabel='IWV MYD    [mm]', 
                                              ylabel='IWV VESPA    [mm]')

(vespaVSmod_scatter_ok*vespaVSmyd_scatter_ok
*slope
*regress_mxd_vespa_sk_text
#*regress_MOD_sk[0]*regress_MYD_sk[0]
*regress_mxd_vespa_sk[0]).opts(title=title, show_grid=True, xlabel= 'IWV MXD    [mm]')

### Stagionalità

In [ ]:
xlim = [0,2]
ylim = [0,2]
xlim = [0,25]
ylim = [0,25]
#x_offset=xlim[1]-30
#y_offset=ylim[0]+5
scale = 5

regress_mod_vespa_s_text = regression_eq_text(new_mod_vespa.IWV_MODIS,
                                        new_mod_vespa.IWV_THAAO, regress_mod_sk,
                                        x_offset=xlim[0]-0.5,y_offset=ylim[0]+0.5)
regress_myd_vespa_s_text = regression_eq_text(new_myd_vespa.IWV_MODIS,
                                        new_myd_vespa.IWV_THAAO, regress_myd_sk,
                                        x_offset=xlim[0]-0.5,y_offset=ylim[0]+0.5)
regress_mxd_vespa_s_text = regression_eq_text(new_mxd_vespa.IWV_MODIS,
                                        new_mxd_vespa.IWV_THAAO, regress_mxd_vespa_sk,
                                        x_offset=xlim[0]-0.5,y_offset=ylim[0]+0.5)

vespaVSmod_scatter = new_mod_vespa.hvplot.scatter(x='IWV_MODIS', y='IWV_THAAO', 
                                              height=height, width=width, 
                                              c='vza', 
                                              xlim=xlim, ylim=ylim, 
                                              s='diff_distance', scale=scale, 
                                              xlabel='IWV MOD    [mm]', 
                                              ylabel='IWV THAAO    [mm]',
                                              groupby=['Time.month'])

vespaVSmyd_scatter = new_myd_vespa.hvplot.scatter(x='IWV_MODIS', y='IWV_THAAO', 
                                              height=height, width=width, c='vza',
                                              xlim=xlim, ylim=ylim, 
                                              s='diff_distance', scale=scale, 
                                              xlabel='IWV MYD    [mm]',
                                              ylabel='IWV THAAO    [mm]',
                                              groupby=['Time.month'])

(vespaVSmod_scatter*slope*regress_mod_sk[0]*regress_mod_vespa_s_text
+ vespaVSmyd_scatter*slope*regress_myd_sk[0]*regress_myd_vespa_s_text
+ (vespaVSmod_scatter*vespaVSmyd_scatter*slope*regress_mxd_vespa_s_text
   *regress_mod_sk[0]*regress_myd_sk[0]*regress_mxd_vespa_sk[0]
  ).opts(title='DDDDD', xlabel= 'IWV MXD    [mm]')).cols(1)

### GroupBy month

In [ ]:
vespaVSmod_scatter = new_mod_vespa.hvplot.scatter(x='IWV_MODIS', y='IWV_THAAO',
                                              height=300,
                                              xlim=xlim, ylim=ylim,
                                              scale=0.3, xlabel='IWV MOD    [mm]',
                                              ylabel='IWV THAAO    [mm]',
                                              by=['Time.month'], groupby=['Time.month'])

vespaVSmyd_scatter = new_myd_vespa.hvplot.scatter(x='IWV_MODIS', y='IWV_THAAO',
                                              height=300, 
                                              xlim=xlim, ylim=ylim,
                                              scale=0.3, xlabel='IWV MYD    [mm]',
                                              ylabel='IWV THAAO    [mm]',
                                              by=['Time.month'], groupby=['Time.month'])

(vespaVSmod_scatter*slope*regress_mod_sk[0]*regress_mod_vespa_s_text
+ vespaVSmyd_scatter*slope*regress_myd_sk[0]*regress_myd_vespa_s_text
+ (vespaVSmod_scatter*vespaVSmyd_scatter*slope*regress_mod_sk[0]*regress_myd_sk[0]
*regress_mxd_vespa_sk[0]*regress_mxd_vespa_s_text
  ).opts(title='MXD vs VESPA IWV seasonality by month', 
#         xlabel = 'IWV MXD (mm)', ylabel='IWV THAAO (mm)')).cols(1)
         xlabel = 'IWV MXD    [mm]', ylabel='IWV THAAO    [mm]')).cols(1)

In [ ]:
vespaVSmod_scatter = new_mod_vespa.hvplot.scatter(x='IWV_MODIS', y='IWV_THAAO',
                                              height=300,
                                              xlim=xlim, ylim=ylim,
                                              scale=0.3, 
                                              xlabel='IWV MOD    [mm]', ylabel='IWV THAAO    [mm]',
                                              by=['Time.month'], groupby=['Time.month'])

vespaVSmyd_scatter = new_myd_vespa.hvplot.scatter(x='IWV_MODIS', y='IWV_THAAO',
                                              height=300, 
                                              xlim=xlim, ylim=ylim,
                                              scale=0.3,
                                              xlabel='IWV MOD    [mm]', ylabel='IWV THAAO    [mm]',
                                              by=['Time.month'], groupby=['Time.month'])

pn.panel((vespaVSmod_scatter*slope*regress_mod_sk[0]*regress_mod_vespa_s_text
+ vespaVSmyd_scatter*slope*regress_myd_sk[0]*regress_myd_vespa_s_text
+ (vespaVSmod_scatter*vespaVSmyd_scatter*slope*regress_mod_sk[0]*regress_myd_sk[0]
*regress_mxd_vespa_sk[0]*regress_mxd_vespa_s_text
  ).opts(title='MXD vs VESPA IWV seasonality animation', 
         xlabel = 'IWV MXD    [mm]', ylabel='IWV THAAO    [mm]')).cols(1),
#         xlabel = 'IWV MXD (mm)', ylabel='IWV THAAO (mm)')).cols(1),
         widget_type='scrubber', widget_location='top')

### By month

In [ ]:
#cmap = 'hsv'
vespaVSmod_scatter = new_mod_vespa.hvplot.scatter(x='IWV_MODIS', y='IWV_THAAO',
                                               height=height, width=width,
#                                              cmap=cmap,
                                              scale=0.3, 
                                              xlabel='IWV MOD    [mm]', ylabel='IWV THAAO    [mm]',
                                              by=['Time.month'])

vespaVSmyd_scatter = new_myd_vespa.hvplot.scatter(x='IWV_MODIS', y='IWV_THAAO',
                                              height=height, width=width,
#                                              cmap=cmap,
                                              scale=0.3, 
                                              xlabel='IWV MOD    [mm]', ylabel='IWV THAAO    [mm]',
                                              by=['Time.month'])

(vespaVSmod_scatter*slope*regress_mod_sk[0]*regress_mod_vespa_s_text
+ vespaVSmyd_scatter*slope*regress_myd_sk[0]*regress_myd_vespa_s_text
+ (vespaVSmod_scatter*vespaVSmyd_scatter*slope*regress_mod_sk[0]*regress_myd_sk[0]
*regress_mxd_vespa_sk[0]*regress_mxd_vespa_sk_text 
  ).opts(title='MXD vs VESPA IWV seasonality', 
         xlabel = 'IWV MXD   [mm]', ylabel='IWV THAAO   [mm]')).cols(1)

In [ ]:
# ok stagionalità
#cmap = 'hsv'
vespaVSmod_scatter = new_mod_vespa.hvplot.scatter(x='IWV_MODIS', y='IWV_THAAO',
                                               height=height, width=width,
#                                              cmap=cmap,
                                              scale=0.3, 
                                              xlabel='IWV MOD    [mm]', ylabel='IWV VESPA    [mm]',
                                              by=['Time.month'])

vespaVSmyd_scatter = new_myd_vespa.hvplot.scatter(x='IWV_MODIS', y='IWV_THAAO',
                                              height=height, width=width,
#                                              cmap=cmap,
                                              scale=0.3, 
                                              xlabel='IWV MOD    [mm]', ylabel='IWV VESPA    [mm]',
                                              by=['Time.month'])


(vespaVSmod_scatter*vespaVSmyd_scatter
# *slope
#*regress_MOD_sk[0]*regress_MYD_sk[0]
*regress_mxd_vespa_sk[0]*regress_mxd_vespa_sk_text
).opts(title='IWV seasonality', 
       xlabel = 'IWV MXD    [mm]', ylabel='IWV THAAO    [mm]', show_grid=True)

In [ ]:
scale = 5
vespaVSmod_scatter = new_mod_vespa.hvplot.scatter(x='vza', y='diff_perc_IWV',
                                              height=height, width=width, 
#                                              c='vza',
#                                              s='diff_distance', scale=scale,
                                              xlabel='vza',
                                              ylabel='difference IWV THAAO'
                                             ) #.hist(num_bins=21,dimension=['IWV_MODIS','IWV_THAAO'])


vespaVSmyd_scatter = new_myd_vespa.hvplot.scatter(x='vza', y='diff_perc_IWV',
                                              height=height, width=width, 
#                                              c='vza',
#                                              s='diff_distance', scale=scale, 
                                              xlabel='vza', 
                                              ylabel='difference IWV THAAO'
                                             ) #.hist(num_bins=21,dimension=['IWV_MODIS','IWV_THAAO'])

#(vespaVSmod_scatter*slope*regress_mod_sk[0]*regress_mod_sk_text
#+ vespaVSmyd_scatter*slope*regress_myd_sk[0]*regress_myd_sk_text
#+ vespaVSmod_scatter*vespaVSmyd_scatter*slope*regress_mxd_vespa_sk_text
# *regress_mod_sk[0]*regress_myd_sk[0]*regress_mxd_vespa_sk[0]).cols(1)
(vespaVSmod_scatter
+ vespaVSmyd_scatter).cols(1)

### Diff IWV MXD-THAAO vs IWV THAAO 

In [ ]:
height=380
width=1000

In [ ]:
new_mod_vespa.head(1)

In [ ]:
Diff_iwv_vespaVSmod_scatter = new_mod_vespa.hvplot.scatter(x='IWV_THAAO',
#                                                   y='diff_IWV', by=['time.month'],
#                                                   xlabel='IWV VESPA    [cm]',
#                                                   ylabel='Diff MOD-VESPA    [cm]',
                                                   y='diff_IWV', by=['Time.month'],
                                                   xlabel='IWV VESPA    [mm]',
                                                   ylabel='Diff MOD-VESPA    [mm]',
                                                   height=height, width=width, 
                                                   scale=scale,grid=True)

Diff_iwv_vespaVSmyd_scatter = new_myd_vespa.hvplot.scatter(x='IWV_THAAO',
#                                                   y='diff_IWV', by=['time.month'],
#                                                   xlabel='IWV VESPA    [cm]',
#                                                   ylabel='Diff MYD-VESPA    [cm]',
                                                   y='diff_IWV', by=['Time.month'],
                                                   xlabel='IWV VESPA    [mm]',
                                                   ylabel='Diff MYD-VESPA    [mm]',
                                                   height=height, width=width, 
                                                   scale=scale,grid=True)

(Diff_iwv_vespaVSmod_scatter
+ Diff_iwv_vespaVSmyd_scatter
+ (Diff_iwv_vespaVSmod_scatter*Diff_iwv_vespaVSmyd_scatter
  ).opts(ylabel='Diff MXD-VESPA    [mm]')).cols(1)

In [ ]:
title = f'IWV MXD-VESPA              deltatime={deltatime}             # data points={num_mxd_vespa_points_deltatime}'
((Diff_iwv_vespaVSmod_scatter*Diff_iwv_vespaVSmyd_scatter).
 opts(title=title, ylabel='Diff MXD-VESPA    [mm]'))

### Diff % IWV MXD-THAAO vs IWV THAAO 

In [ ]:
Diffperc_iwv_vespaVSmod_scatter = new_mod_vespa.hvplot.scatter(x='IWV_THAAO',
                                                   y='diff_perc_IWV', by=['Time.month'],
                                                   xlabel='IWV VESPA    [mm]',
                                                   ylabel='100*(MOD-VESPA)/VESPA',
                                                   height=height, width=width, 
                                                   scale=scale,grid=True)

Diffperc_iwv_vespaVSmyd_scatter = new_myd_vespa.hvplot.scatter(x='IWV_THAAO',
                                                   y='diff_perc_IWV', by=['Time.month'],
                                                   xlabel='IWV VESPA    [mm]',
                                                   ylabel='100*(MYD-VESPA)/VESPA',
                                                   height=height, width=width, 
                                                   scale=scale,grid=True)

(Diffperc_iwv_vespaVSmod_scatter
+ Diffperc_iwv_vespaVSmyd_scatter
+ (Diffperc_iwv_vespaVSmod_scatter*Diffperc_iwv_vespaVSmyd_scatter)
 .opts(ylabel='100*(MXD-VESPA)/VESPA')).cols(1)

In [ ]:
title = f'IWV 100*(MXD-VESPA)/VESPA              deltatime={deltatime}             # data points={num_mxd_vespa_points_deltatime}'      # data points={number_MXD_data_points_deltatime}'
((Diffperc_iwv_vespaVSmod_scatter*Diffperc_iwv_vespaVSmyd_scatter)
 .opts(title=title, ylabel='100*(MXD-VESPA)/VESPA'))